In [32]:
import openai
import os
import datetime
from openai import OpenAI
import argparse
import pandas as pd
import json
import re
from tqdm import tqdm
import csv

# parse the prediction
def segement_predict(predict):

    prediction = predict.replace('\n\n','\n')
    pattern = r'(\d+)\.\s*([^\d\n]+(?:\n(?!\d+\.)[^\d\n]+)*)'
    diagnosis_list = re.findall(pattern, prediction)

    # if len(processed_list) < 10:
    #     processed_list = prediction.split('\n')

    if len(diagnosis_list) > 10:
            diagnosis_list = diagnosis_list[-10:]

    return diagnosis_list


In [33]:
name = 'HuatuoGPT-o1-70B_JAMAFinalAll'

# HuatuoGPT-o1-70B_JAMAFinalAll
# HuatuoGPT-o1-7B_JAMAFinalAll

file_name = f'/home/gy237/project/llama3/total_final_test/Llama3.1_final_test/{name}.jsonl'

save_path = file_name.split('.jsonl')[0]


data = []
with open(file_name, 'r', encoding='utf-8') as file:
    for line in file:
        item = json.loads(line.strip())
        data.append(item)


acc_dic = {}
error_list = []
for i in data:
    diagnosis = i['true']
    predict = i['predict']

    diagnosis_list = segement_predict(predict)

    processed_list = [str(number) + '. ' + diagnosis.strip() for number, diagnosis in diagnosis_list]
    numbers = [number for number, diagnosis in diagnosis_list]
    i['predict_list'] = processed_list

    if len(numbers) < 10:
        error_list.append({'predict': i['predict'], 'predict_list': i['predict_list']})
    elif numbers[0] != '1' or numbers[-1] != '10':
        error_list.append({'predict': i['predict'], 'predict_list': i['predict_list']})


print(f'Number of Error list: {len(error_list)}')
with open(f'{save_path}_error.json', 'w', encoding='utf-8') as file:
    json.dump(error_list, file, ensure_ascii=False, indent=4)

Number of Error list: 102
